In [1]:
import pandas as pd
from IPython.display import Image
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from pydna.dseqrecord import Dseqrecord

### SPs

In [16]:
sp_df = pd.read_csv('../data/15_SecretoGen/Best_signal_peptides_for_A_oryzae_RFP_his_tag_sorted.csv')

# Rename the 'Unnamed: 0' column and add prefix to the numbers
#sp_df = sp_df.rename(columns={'Unnamed: 0': 'SP_ID'})
#sp_df['SP_ID'] = sp_df['SP_ID'].apply(lambda x: f'SP_{x}')
sp_df = sp_df.drop(columns=['Unnamed: 0'])
sp_df

,SP_ID,Sequence,Perplexity
0,SP_997,MRPTLLALGVISFALTLHS,7.515804
1,SP_944,MKFTLIILIIVILTIIFSPGALA,7.421489
2,SP_948,MKVTLSLLAVFLAALSASAIPANG,7.179219
3,SP_993,MKAIIILLLIILTLALTVQG,7.140379
4,SP_204,MRVFSATAILALSPLLIASASP,6.925975
...,...,...,...
995,SP_656,MVLGILTITAIPTVTGLANA,1.597785
996,SP_696,MVLGLLTIAAIPTVTGVAQA,1.563614
997,SP_757,MVLGLLTIAAIPTVTGVGNAVSA,1.554127
998,SP_253,MVLGILTITAIPTVTGVAQA,1.535361


In [17]:
#sp_df.to_csv('../data/15_SecretoGen/Best_signal_peptides_for_A_oryzae_RFP_his_tag_sorted.csv', index= False)

In [18]:
sp_df

,SP_ID,Sequence,Perplexity
0,SP_997,MRPTLLALGVISFALTLHS,7.515804
1,SP_944,MKFTLIILIIVILTIIFSPGALA,7.421489
2,SP_948,MKVTLSLLAVFLAALSASAIPANG,7.179219
3,SP_993,MKAIIILLLIILTLALTVQG,7.140379
4,SP_204,MRVFSATAILALSPLLIASASP,6.925975
...,...,...,...
995,SP_656,MVLGILTITAIPTVTGLANA,1.597785
996,SP_696,MVLGLLTIAAIPTVTGVAQA,1.563614
997,SP_757,MVLGLLTIAAIPTVTGVGNAVSA,1.554127
998,SP_253,MVLGILTITAIPTVTGVAQA,1.535361


In [21]:
# Adding the length of sequences to the dataframe
sp_df['Length'] = sp_df['Sequence'].apply(len)

# Removing sequences with length over 26
filtered_df = sp_df[sp_df['Length'] <= 26]
filtered_df

,SP_ID,Sequence,Perplexity,Length
0,SP_997,MRPTLLALGVISFALTLHS,7.515804,19
1,SP_944,MKFTLIILIIVILTIIFSPGALA,7.421489,23
2,SP_948,MKVTLSLLAVFLAALSASAIPANG,7.179219,24
3,SP_993,MKAIIILLLIILTLALTVQG,7.140379,20
4,SP_204,MRVFSATAILALSPLLIASASP,6.925975,22
...,...,...,...,...
995,SP_656,MVLGILTITAIPTVTGLANA,1.597785,20
996,SP_696,MVLGLLTIAAIPTVTGVAQA,1.563614,20
997,SP_757,MVLGLLTIAAIPTVTGVGNAVSA,1.554127,23
998,SP_253,MVLGILTITAIPTVTGVAQA,1.535361,20


In [22]:

# Getting the top 30
top_30 = sp_df.head(30)

# Sampling the remaining dataframe for a good mix
remaining_sample_size = 84 - 30
remaining_sample = sp_df.iloc[30:].sample(n=remaining_sample_size, random_state=1)

# Combining the top 30 with the remaining sample
final_sample = pd.concat([top_30, remaining_sample]).reset_index(drop=True)
final_sample

,SP_ID,Sequence,Perplexity,Length
0,SP_997,MRPTLLALGVISFALTLHS,7.515804,19
1,SP_944,MKFTLIILIIVILTIIFSPGALA,7.421489,23
2,SP_948,MKVTLSLLAVFLAALSASAIPANG,7.179219,24
3,SP_993,MKAIIILLLIILTLALTVQG,7.140379,20
4,SP_204,MRVFSATAILALSPLLIASASP,6.925975,22
...,...,...,...,...
79,SP_952,MRVSTIPTALLGLSLSAVNA,3.271387,20
80,SP_659,MVRLNSITALASLLAVSLTAA,3.640410,21
81,SP_576,MILSYRSLVVTTVALLGLSTPATA,3.873739,24
82,SP_560,MVLLKPLLATALLASLTPAVA,3.045672,21


In [45]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from pydna.dseq import Dseq

# Simplified back-translation codon table
amino_acid_to_codon = {
    'A': 'GCT', 'R': 'CGT', 'N': 'AAT', 'D': 'GAT',
    'C': 'TGT', 'Q': 'CAA', 'E': 'GAA', 'G': 'GGT',
    'H': 'CAT', 'I': 'ATT', 'L': 'CTT', 'K': 'AAA',
    'M': 'ATG', 'F': 'TTT', 'P': 'CCT', 'S': 'TCT',
    'T': 'ACT', 'W': 'TGG', 'Y': 'TAT', 'V': 'GTT',
    'X': 'NNN'  # 'X' represents any amino acid
}

def back_translate(protein_sequence):
    return ''.join([amino_acid_to_codon[aa] for aa in protein_sequence])


In [46]:

seq_records = []
for index, row in final_sample.iterrows():
    dna_sequence = back_translate(row['Sequence'])
    dseq_obj = Dseq(dna_sequence)
    dseqrecord = Dseqrecord(dseq_obj, name=row['SP_ID'], description=f"Perplexity: {row['Perplexity']}, Length: {row['Length']}")
    seq_records.append(dseqrecord)

seq_records

[Dseqrecord(-57),
 Dseqrecord(-69),
 Dseqrecord(-72),
 Dseqrecord(-60),
 Dseqrecord(-66),
 Dseqrecord(-57),
 Dseqrecord(-69),
 Dseqrecord(-60),
 Dseqrecord(-60),
 Dseqrecord(-75),
 Dseqrecord(-66),
 Dseqrecord(-63),
 Dseqrecord(-54),
 Dseqrecord(-51),
 Dseqrecord(-66),
 Dseqrecord(-57),
 Dseqrecord(-63),
 Dseqrecord(-63),
 Dseqrecord(-57),
 Dseqrecord(-51),
 Dseqrecord(-57),
 Dseqrecord(-84),
 Dseqrecord(-66),
 Dseqrecord(-54),
 Dseqrecord(-60),
 Dseqrecord(-63),
 Dseqrecord(-57),
 Dseqrecord(-54),
 Dseqrecord(-60),
 Dseqrecord(-60),
 Dseqrecord(-66),
 Dseqrecord(-57),
 Dseqrecord(-57),
 Dseqrecord(-63),
 Dseqrecord(-72),
 Dseqrecord(-66),
 Dseqrecord(-51),
 Dseqrecord(-63),
 Dseqrecord(-63),
 Dseqrecord(-63),
 Dseqrecord(-54),
 Dseqrecord(-60),
 Dseqrecord(-66),
 Dseqrecord(-60),
 Dseqrecord(-72),
 Dseqrecord(-60),
 Dseqrecord(-63),
 Dseqrecord(-66),
 Dseqrecord(-57),
 Dseqrecord(-60),
 Dseqrecord(-60),
 Dseqrecord(-63),
 Dseqrecord(-60),
 Dseqrecord(-63),
 Dseqrecord(-72),
 Dseqrecor

In [51]:
a_oryzae ={
    'F': {'TTT': 0.38, 'TTC': 0.62},
    'S': {'TCT': 0.18, 'TCC': 0.20, 'TCA': 0.14, 'TCG': 0.16, 'AGT': 0.13, 'AGC': 0.18},
    'Y': {'TAT': 0.47, 'TAC': 0.53},
    'C': {'TGT': 0.46, 'TGC': 0.54},
    '*': {'TAA': 0.33, 'TAG': 0.29, 'TGA': 0.38},
    'L': {'TTA': 0.07, 'TTG': 0.18, 'CTT': 0.19, 'CTC': 0.23, 'CTA': 0.11, 'CTG': 0.22},
    'W': {'TGG': 1.00},
    'P': {'CCT': 0.27, 'CCC': 0.26, 'CCA': 0.25, 'CCG': 0.22},
    'H': {'CAT': 0.53, 'CAC': 0.47},
    'Q': {'CAA': 0.43, 'CAG': 0.57},
    'R': {'CGT': 0.18, 'CGC': 0.23, 'CGA': 0.17, 'CGG': 0.18, 'AGA': 0.13, 'AGG': 0.12},
    'I': {'ATT': 0.36, 'ATC': 0.50, 'ATA': 0.14},
    'M': {'ATG': 1.00},
    'T': {'ACT': 0.24, 'ACC': 0.32, 'ACA': 0.23, 'ACG': 0.20},
    'N': {'AAT': 0.45, 'AAC': 0.55},
    'K': {'AAA': 0.36, 'AAG': 0.64},
    'V': {'GTT': 0.27, 'GTC': 0.33, 'GTA': 0.13, 'GTG': 0.27},
    'A': {'GCT': 0.27, 'GCC': 0.30, 'GCA': 0.23, 'GCG': 0.20},
    'D': {'GAT': 0.53, 'GAC': 0.47},
    'E': {'GAA': 0.44, 'GAG': 0.56},
    'G': {'GGT': 0.28, 'GGC': 0.31, 'GGA': 0.24, 'GGG': 0.17}
}


In [58]:
from dnachisel import DnaOptimizationProblem, CodonOptimize

# Codon optimization for Aspergillus oryzae
codon_optimized_records = []
for seq in seq_records:
    problem = DnaOptimizationProblem(
        sequence=str(seq.seq),
        constraints=[],
        objectives=[CodonOptimize(codon_usage_table=a_oryzae, method='use_best_codon')], 
    )
    problem.optimize()
    optimized_dna_sequence = str(problem.sequence)
    
    dseq_obj = Dseq(optimized_dna_sequence)
    print(dseq_obj)
    dseqrecord = Dseqrecord(dseq_obj, name=row['SP_ID'], description=f"Perplexity: {row['Perplexity']}, Length: {row['Length']}")
    codon_optimized_records.append(dseqrecord)


ATGCGCCCTACCCTCCTCGCCCTCGGCGTCATGTCCTTCGCCCTCACCCTCCATTCC


ATGAACTTCACCCTCATGATGCTCATGATGGTCATGCTCACCATGATGTTCTCCCCTGGCGCCCTCGCC


ATGAACGTCACCCTCTCCCTCCTCGCCGTCTTCCTCGCCGCCCTCTCCGCCTCCGCCATGCCTGCCAAGGGC


ATGAACGCCATGATGATGCTCCTCCTCATGATGCTCACCCTCGCCCTCACCGTCCAGGGC


ATGCGCGTCTTCTCCGCCACCGCCATGCTCGCCCTCTCCCCTCTCCTCATGGCCTCCGCCTCCCCT


...

ATGCGCGTCTCCACCATGCCTACCGCCCTCCTCGGCCTCTCCCTCTCCGCCGTCAAGGCC


ATGGTCCGCCTCAAGTCCATGACCGCCCTCGCCTCCCTCCTCGCCGTCTCCCTCACCGCCGCC


ATGATGCTCTCCTACCGCTCCCTCGTCGTCACCACCGTCGCCCTCCTCGGCCTCTCCACCCCTGCCACCGCC


ATGGTCCTCCTCAACCCTCTCCTCGCCACCGCCCTCCTCGCCTCCCTCACCCCTGCCGTCGCC


ATGGTCCATCTCAACACCCTCGCCCTCGCCGTCTCCGCCCTCCTCACCCCTTCCATGGTCTCCGCC


In [55]:
codon_optimized_records

[Dseqrecord(-57),
 Dseqrecord(-69),
 Dseqrecord(-72),
 Dseqrecord(-60),
 Dseqrecord(-66),
 Dseqrecord(-57),
 Dseqrecord(-69),
 Dseqrecord(-60),
 Dseqrecord(-60),
 Dseqrecord(-75),
 Dseqrecord(-66),
 Dseqrecord(-63),
 Dseqrecord(-54),
 Dseqrecord(-51),
 Dseqrecord(-66),
 Dseqrecord(-57),
 Dseqrecord(-63),
 Dseqrecord(-63),
 Dseqrecord(-57),
 Dseqrecord(-51),
 Dseqrecord(-57),
 Dseqrecord(-84),
 Dseqrecord(-66),
 Dseqrecord(-54),
 Dseqrecord(-60),
 Dseqrecord(-63),
 Dseqrecord(-57),
 Dseqrecord(-54),
 Dseqrecord(-60),
 Dseqrecord(-60),
 Dseqrecord(-66),
 Dseqrecord(-57),
 Dseqrecord(-57),
 Dseqrecord(-63),
 Dseqrecord(-72),
 Dseqrecord(-66),
 Dseqrecord(-51),
 Dseqrecord(-63),
 Dseqrecord(-63),
 Dseqrecord(-63),
 Dseqrecord(-54),
 Dseqrecord(-60),
 Dseqrecord(-66),
 Dseqrecord(-60),
 Dseqrecord(-72),
 Dseqrecord(-60),
 Dseqrecord(-63),
 Dseqrecord(-66),
 Dseqrecord(-57),
 Dseqrecord(-60),
 Dseqrecord(-60),
 Dseqrecord(-63),
 Dseqrecord(-60),
 Dseqrecord(-63),
 Dseqrecord(-72),
 Dseqrecor